In [ ]:
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
from ukbb_recessive.regression.regressions import read_results_excel
import ukbb_recessive.regression.plotting as plotting
from matplotlib.cm import get_cmap


sns.set_style("whitegrid")

# Add fonts
plotting.add_fonts(['../../../data/fonts'])
plotting.configure_matplotlib()

PLOTS_OUTPUT_DIR = '../../../data/plots'
tables_folder = '../../../data/tables'

# Prepare datasets

## Load PLP-based estimations

In [ ]:
# Dictionary to rename target columns
renaming_dict_target = {
    'is_blond': 'Hair color', 
    'years_of_edu': 'Years of\neducation',
    'diagnosis_total_ICD10_cnt_log': 'Diagnoses\ncount (log)',
    'ICD_infertility': 'Infertility',
    'living_with_a_partner': 'Has partner',
    'fluid_intelligence_score': 'Fluid\nintelligence',
    'childlessness': 'Childlessness'
}

use_dataset = 'Roulette'

results_path = f"{tables_folder}/table_id_vs_rest_analysis.xlsx"

# load flat table
id_vs_rest = read_results_excel(results_path, flatten_multiindex=True)

# leave only necessary s_het effect
id_vs_rest = id_vs_rest[id_vs_rest['feature'].str.contains("s_het")]

# add odds ratio distance
id_vs_rest = plotting.add_odds_ratio_intervals(id_vs_rest)
id_vs_rest = plotting.add_effect_size_intervals(id_vs_rest)

# filter out dataset
id_vs_rest = id_vs_rest[id_vs_rest['dataset'] == 'Roulette']


# select targets of interest
id_vs_rest = id_vs_rest[id_vs_rest['target'].isin(renaming_dict_target.keys())].copy()
id_vs_rest['target'] = id_vs_rest['target'].apply(lambda x: renaming_dict_target.get(x, x))

# select gender of interest
id_vs_rest = id_vs_rest[id_vs_rest['gender'] == 'all']

# select analyses
renaming_dict_feature = {
    's_het_recessive_AR_without_ID': 'Other recessive genes',
	's_het_recessive_ID_total': 'Recessive ID genes',
}

id_vs_rest = id_vs_rest[id_vs_rest['feature'].isin(renaming_dict_feature.keys())].copy()


id_vs_rest.loc[:, 'analysis'] = id_vs_rest['feature'].apply(lambda x: renaming_dict_feature.get(x, x))


print("Target:", id_vs_rest['target'].unique())

id_vs_rest.head(3)

## Load synonymous-based estimations

In [ ]:
results_path = f"{tables_folder}/sampling/synonymous/table_synonymous_id_vs_rest_analysis.xlsx"

# load flat table
synonymous = read_results_excel(results_path, flatten_multiindex=True)

# leave only necessary s_het effect
synonymous = synonymous[synonymous['feature'].str.contains("s_het")]

# add odds ratio distance
synonymous = plotting.add_odds_ratio_intervals(synonymous)
synonymous = plotting.add_effect_size_intervals(synonymous)

# select targets of interest
synonymous = synonymous[synonymous['target'].isin(renaming_dict_target.keys())].copy()
synonymous['target'] = synonymous['target'].apply(lambda x: renaming_dict_target.get(x, x))

# select gender of interest
synonymous = synonymous[synonymous['gender'] == 'all']

synonymous.head(3)

# Plot

In [ ]:
def plot_synonymous(ax, targets, synonymous, phenotypes, plot_entity='odds_ratio', vline_loc=1):

    # select necessary targets
    data_syns = synonymous[synonymous['target'].isin(targets)].copy()
    data_plps = phenotypes[phenotypes['target'].isin(targets)].copy()

    # sort dataset according to the target order
    data_syns.target = pd.Categorical(data_syns.target, categories=targets)
    data_syns = data_syns.sort_values('target')

    data_plps.target = pd.Categorical(data_plps.target, categories=targets)
    data_plps = data_plps.sort_values('target')

    # plot neutral line
    ax.vlines(x=vline_loc, ymin=-3, ymax=10, color='grey', linestyle='--', alpha=0.5, linewidth=0.5)
    
    vp = sns.violinplot(data=data_syns, y='target', x=plot_entity, hue='target',
                        ax=ax, linewidth=0.5, linecolor='k', palette=['#a0b4ff'])

    # plot PLPs estimation
    pp = sns.pointplot(data=data_plps, y='target', x=plot_entity, linestyle='none', markers='o', 
                       ax=ax, color='#ffa260', markersize=3, markeredgewidth=0.5, markeredgecolor='k')

In [ ]:
import matplotlib.ticker as ticker

cm = 1/2.54  # centimeters in inches
k = 1.
fig = plt.figure(constrained_layout=True, figsize=(8.8*cm*k, 5*cm*k))

# divide subfigure into left and right parts
subfigs = fig.subfigures(nrows=1, ncols=2, width_ratios=[1, 1], wspace=0.0)

targets = ['Childlessness', 'Years of\neducation', 'Fluid\nintelligence', 'Diagnoses\ncount (log)',  'Hair color']

# plot phenotypes results
ax_left = subfigs[0].subplots(1, 1)


plot_synonymous(ax_left, synonymous=synonymous[synonymous['feature'] == 's_het_recessive_ID_total'], 
                phenotypes=id_vs_rest[id_vs_rest['feature'] == 's_het_recessive_ID_total'], 
                targets=targets, plot_entity='effect', vline_loc=0)

ax_left.set_title('Recessive ID genes')
plotting.configure_axis(ax_left, x_label='Effect size', y_label=None, xlim=(-4.1, 4.1), ymargin=0.02, format_x=True)

ax_left.set_xticks([-4., -2., 0., 2., 4.])
ax_left.set_xticklabels([-4., -2., 0., 2., 4.])

# plot synonymous
ax_right = subfigs[1].subplots(1, 1)


plot_synonymous(ax_right, synonymous=synonymous[synonymous['feature'] == 's_het_recessive_AR_without_ID'], 
                phenotypes=id_vs_rest[id_vs_rest['feature'] == 's_het_recessive_AR_without_ID'], 
                targets=targets, plot_entity='effect', vline_loc=0)
ax_right.set_title('Other recessive genes')

ax_right.set_xticks([-1., -.5, 0., .5, 1.])
ax_right.set_xticklabels([-1., -.5, 0., .5, 1.])

plotting.configure_axis(ax_right, x_label='Effect size', y_label=None, xlim=(-1.3, 1.3), ymargin=0.02, format_x=True)

# save figure
plt.savefig(f"{PLOTS_OUTPUT_DIR}/figure_4.pdf", format="pdf", bbox_inches="tight")